### 1. dictionary packing and unpacking

```
>>> def example(**kwargs):
...     return kwargs.keys()
...
>>> example(a=10, b=20, c=[0, 0, 0])
['a', 'c', 'b']
```

In [11]:
import numpy as np
criteria = '{3sigma}>{spec}'
dct = {'3sigma': 1.9,'AI_CD': 'nan','spec': 1.600,}
dct2 = {'3sigma': 'nan','AI_CD': 'nan','spec': 1.600,}
print eval(criteria.format(**dct), {}, {'nan':np.nan})
print eval(criteria.format(**dct2), {}, {'nan':np.nan})

True
False


In [9]:
criteria = '{sigma}>{spec}'
print eval(criteria.format(sigma=1.9, AI_CD='nan', spec=1.6), {}, {'nan':np.nan})

True


### 2. built-in functions {compile, eval, exec}, {map, filter}, hash, repr, 

1. `basestring()`
    - isinstance(obj, basestring) is equivalent to isinstance(obj, (str, unicode)).
    
2. `compile`, `exec`, `eval`:
    - `compile(source, filename, mode[, flags[, dont_inherit]])`, output into a code or AST obj. Code can be executed by 'exec' or evaluated by 'eval'.
        ```
        >>> codeobj = compile('x = 2\nprint "X is", x', 'fakemodule', 'exec')
        >>> exec(codeobj)
        X is 2
        ```
        For instance, if you have a Python program that allows users to script its behavior with custom Python code, you might use compile and exec to store and execute these user-defined scripts.
        Risk is to introduce some unsafe code. (For instance, imagine that in my example above the code was `formatYourHardDrive()` instead of `print x`.)
     
    - The difference of this three: exec({statement}), eval({expression}), compile to speed up repeated invocations like execute and evaluate
        ```
        >>> a = 5
        >>> eval('37 + a')   # it is an expression
        42
        >>> exec('37 + a')   # it is an expression statement
        >>> exec('a = 47')   # modify a global variable as a side effect
        >>> a
        47
        >>> eval('a = 47')  # you cannot evaluate a statement
        Traceback (most recent call last):
          File "<stdin>", line 1, in <module>
          File "<string>", line 1
            a = 47
              ^
        SyntaxError: invalid syntax
        ```  
3. `map`, `filter`
4. 'dict', list, zip(), iterable, mapping or **kwarg(key-word arugments)
5. `hash`: Return the hash value of the object (if it has one). They are used to quickly compare dictionary keys during a dictionary lookup.
6. `repr` and `str`: `repr` is debugging and pass the string into `eval`, `str` is for variable type conversion.

In [12]:
codeobj = compile('x = 2\nprint "X is", x', 'fakemodule', 'exec')
exec(codeobj)

codeobj = compile(r'x+2', '', 'eval')
print eval(codeobj)
eval('x + 3')


X is 2
4


5

In [33]:
print filter(lambda x: x>0, xrange(-4,5))
print map(lambda x: str(x), xrange(-4,5))
s = 'AG1=0:300 , Ap =20:300,MGS=30:300,default=20:300'
import string
sigma_ranges = map(string.strip, s.split(','))
print 'step 1', sigma_ranges
sigma_ranges = map(lambda x: map(string.strip, x.split('=')), sigma_ranges)
print 'step 2', sigma_ranges
sigma_ranges = dict(sigma_ranges)
print 'step 3', sigma_ranges
print hash('MGS')

[1, 2, 3, 4]
['-4', '-3', '-2', '-1', '0', '1', '2', '3', '4']
step 1 ['AG1=0:300', 'Ap =20:300', 'MGS=30:300', 'default=20:300']
step 2 [['AG1', '0:300'], ['Ap', '20:300'], ['MGS', '30:300'], ['default', '20:300']]
step 3 {'MGS': '30:300', 'Ap': '20:300', 'default': '20:300', 'AG1': '0:300'}
-29093136


3.1 tricks

1. get a default value if a key is not exist


In [36]:
vAG = sigma_ranges.get('AG1', sigma_ranges.get('default', '0:300'))
print vAG

0:300


In [22]:
import string
spec = "FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0001.tif;EDGE_FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0001.txt;DETECT_EDGE:2;ALIGN_EDGE:0;XAXIS_SWAP:0;YAXIS_SWAP:0;ROTATION:0;PIXEL:0.96500000;CENTER_X:-11139750.00000000;CENTER_Y:7135092.00000000;THETA:0.00410000;SCALE_XY_RATIO:1.00000000;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1"
specrow = map(string.strip, spec.split(";"))
print specrow
specrow = [kw.split(":") for kw in specrow]
print specrow
specrow = {kw[0]:kw[1] for kw in specrow}
# print specrow

SyntaxError: invalid syntax (<ipython-input-22-c5c496d0746f>, line 5)

NameError: name 'unzip' is not defined

In [68]:
import re
m = re.match(r'^Load SEM image ([/\-\w]+)\.tif', "Load SEM image /gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0000.tif")
m  = re.match(r'Aligned Spec : ([^=@%]+)$', "Aligned Spec : FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0000.tif;EDGE_FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0000.txt;DETECT_EDGE:2;ALIGN_EDGE:0;XAXIS_SWAP:0;YAXIS_SWAP:0;ROTATION:0;PIXEL:0.96500000;CENTER_X:-11139696.00000000;CENTER_Y:7135032.00000000;THETA:0.00420000;SCALE_XY_RATIO:1.00000000;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1")
print m.group()
specrow = m.group(1)
specrow = map(string.strip, spec.split(";"))
specrow = [kw.split(":") for kw in specrow]
keys, values= zip(*specrow)
specrow = {kw[0]:kw[1] for kw in specrow}
print keys
print specrow

Aligned Spec : FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0000.tif;EDGE_FILE:/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0000.txt;DETECT_EDGE:2;ALIGN_EDGE:0;XAXIS_SWAP:0;YAXIS_SWAP:0;ROTATION:0;PIXEL:0.96500000;CENTER_X:-11139696.00000000;CENTER_Y:7135032.00000000;THETA:0.00420000;SCALE_XY_RATIO:1.00000000;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1;VR_XINI:-1
('FILE', 'EDGE_FILE', 'DETECT_EDGE', 'ALIGN_EDGE', 'XAXIS_SWAP', 'YAXIS_SWAP', 'ROTATION', 'PIXEL', 'CENTER_X', 'CENTER_Y', 'THETA', 'SCALE_XY_RATIO', 'VR_XINI', 'VR_XINI', 'VR_XINI', 'VR_XINI')
{'VR_XINI': '-1', 'DETECT_EDGE': '2', 'XAXIS_SWAP': '0', 'YAXIS_SWAP': '0', 'EDGE_FILE': '/gpfs/PEG/FEM/peyang/data/iCal/IMEC_SN2/4HMICD/iCal_check_308_fromQian_v2/h/data/dummydb/calibrate/job1/sem/c1/0001.txt', 'CENTER_X': '-11139750.00000000', 'CENTER_Y': '7135092.00000000', 'FILE': '/gpfs/PEG/FEM/peyang/data/